 # [Module 1.7] AutoGluon 훈련 및 추론

이 노트북은 아래와 같은 작업을 합니다.
- 데이터 로딩
- AutoGluon 데이터 타입으로 변경
- AutoGluon 훈련
- 테스트 데이터 로딩 및 추론
- 모델 평가
- 피쳐 Importance 생성

#### AutoGluon Tabular Prediction
- 오토글루온 Tabular 공식 페이지 입니다. 자세한 내용은 아래 링크를 참조 하세요.
    - https://autogluon.mxnet.io/stable/tutorials/tabular_prediction/index.html

## 데이터 세트 로딩
로컬에 저장된 파일을 로딩해서, 다시 한번 Shape(Row, column)을 확인하고 데이터를 확인 합니다.

In [1]:
import pandas as pd

In [2]:
%store -r no_auto_train_file
%store -r no_auto_test_file

In [3]:
train_pd = pd.read_csv(no_auto_train_file)
test_pd = pd.read_csv(no_auto_test_file)

In [4]:
print("auto_train_file: ", train_pd.shape)
print("auto_train_file: ", test_pd.shape)

auto_train_file:  (53740, 18)
auto_train_file:  (13436, 18)


In [5]:
from IPython.display import display as dp
dp(train_pd.head(2))
dp(test_pd.head(2))

,classes,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
0,3,12030,taubate,SP,29.99,10.96,9000.0,fashion_shoes,13481,limeira,SP,1,4,10,SP_SP,taubate_limeira,12030_13481,2640.0
1,2,78075,cuiaba,MT,23.90,26.82,1500.0,furniture_decor,17051,bauru,SP,1,4,10,MT_SP,cuiaba_bauru,78075_17051,14625.0


,classes,customer_zip_code_prefix,customer_city,customer_state,price,freight_value,product_weight_g,product_category_name_english,seller_zip_code_prefix,seller_city,seller_state,order_weekday,order_day,order_month,customer_seller_state,customer_seller_city,customer_seller_zip_code_prefix,product_volume
0,3,89219,joinville,SC,105.00,23.89,1000.0,watches_gifts,21840,rio de janeiro,RJ,1,19,6,SC_RJ,joinville_rio de janeiro,89219_21840,7632.0
1,0,9950,diadema,SP,99.97,15.80,650.0,housewares,32677,betim,MG,1,19,6,SP_MG,diadema_betim,9950_32677,9600.0


#### 에러 발생시 아래 처럼 수행
1. pip install autogluon
2. pip install --upgrade mxnet
3. from autogluon.tabular import TabularPrediction -> from autogluon import TabularPrediction 수정(실제로는 deprecate 된 api임)

In [6]:
import autogluon.core as ag
from autogluon.tabular import TabularPrediction as task

# from autogluon import TabularPrediction as task

## 오토글루온 데이터 타입으로 변경
로컬에 저장된 파일을 오토글로온의 Tabular 데이터 타입으로 로딩 합니다. 이후 랜덤 샘플링을 하고 데이터를 확인 합니다.

In [7]:
train_data = task.Dataset(file_path= no_auto_train_file)
# subsample_size = train_data.shape[0]
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
print(train_data.head(2))

       classes  customer_zip_code_prefix customer_city customer_state   price  \
588          2                     83260      matinhos             PR   59.99   
45386        3                     84010  ponta grossa             PR  120.00   

       freight_value  product_weight_g product_category_name_english  \
588            18.00             525.0                sports_leisure   
45386          18.72            1300.0     construction_tools_lights   

       seller_zip_code_prefix            seller_city seller_state  \
588                     13456  santa barbara d'oeste           SP   
45386                   13327                  salto           SP   

       order_weekday  order_day  order_month customer_seller_state  \
588                0         30            1                 PR_SP   
45386              4         27            4                 PR_SP   

                 customer_seller_city customer_seller_zip_code_prefix  \
588    matinhos_santa barbara d'oeste          

### 레이블 컬럼 지정
**레이블 컬럼을 지정하여, 오토글루온이 이 컬럼을 레이블로 인식하게 합니다.**

In [8]:
label_column = 'classes'
print("Summary of class variable: \n", train_data[label_column].describe())

Summary of class variable: 
 count    500.000000
mean       2.232000
std        1.151011
min        0.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        4.000000
Name: classes, dtype: float64


## 오토글루운으로 훈련
- 시간 제약을 2시간으로 합니다. 하지만 아래 훈련은 약 5분 소요 됩니다.
- 하지만 presets를 best_quality 으로 설정하면, 시간은 약 1시간 30분이 소요 됩니다. 하지만 이 경우에는 성능은 다소 올라갑니다. 다른 데이터 세트의 경우는 성능이 올라가는 정도가 다를 수 있습니다. 

In [9]:

%%time
dir = 'agModels-predictClass'  # specifies folder where to store trained models
time_limits = 7200 # seconds
# presets = 'best선_quality'
presets = 'medium_quality_faster_train'

predictor = task.fit(train_data=train_data, label=label_column, 
                     output_directory=dir,
                     time_limits=time_limits,
                     presets=presets)



Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to agModels-predictClass/
AutoGluon Version:  0.0.15b20201122
Train Data Rows:    500
Train Data Columns: 17
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	5 unique label values:  [2, 3, 1, 0, 4]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 5
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
Note: NumExpr detected 36 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
	Available Memory:                    71225.78 MB
	Train Data (Original)  Memory Usage: 0.3 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_i

CPU times: user 3min 16s, sys: 11.7 s, total: 3min 28s
Wall time: 26 s


## 테스트 데이터 로딩
- 테스트 데이터를 로딩하여 모델을 평가 합니다.
- 여기서는 y_test의 레이블만 가지고 있는 변수를 생성
- test_data_nolab는 레이블만 없는 피쳐만으로 구성된 데이터를 생성 합니다.

In [10]:
test_data = task.Dataset(file_path= no_auto_test_file)
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(labels=[label_column],axis=1)  # delete label column to prove we're not cheating
print(test_data_nolab.head(2))

Loaded data from: preproc_data/auto_no_fe/test/test.csv | Columns = 18 / 18 | Rows = 13436 -> 13436


   customer_zip_code_prefix customer_city customer_state   price  \
0                     89219     joinville             SC  105.00   
1                      9950       diadema             SP   99.97   

   freight_value  product_weight_g product_category_name_english  \
0          23.89            1000.0                 watches_gifts   
1          15.80             650.0                    housewares   

   seller_zip_code_prefix     seller_city seller_state  order_weekday  \
0                   21840  rio de janeiro           RJ              1   
1                   32677           betim           MG              1   

   order_day  order_month customer_seller_state      customer_seller_city  \
0         19            6                 SC_RJ  joinville_rio de janeiro   
1         19            6                 SP_MG             diadema_betim   

  customer_seller_zip_code_prefix  product_volume  
0                     89219_21840          7632.0  
1                      9950_32677 

## 모델 예측
- 생성된 모델을 통해서 추론(예측)을 합니다.
- 진실값 (y_test), 예측값(prediction)을 제공하여 accuracy, f1, precision, recall 를 계산 합니다.

In [13]:
type(test_data_nolab)

autogluon.tabular.task.tabular_prediction.dataset.TabularDataset

In [14]:
def change_type(raw_df):
    df = raw_df.copy()
    int_columns = df.columns[df.dtypes == 'int64'].tolist()
    for e in int_columns:
        print(e)
        df[e] = df[e].astype('float64')
#    int_index = [num -1 for num in int_index]

    

    return df
    
test2_data_nolab = change_type(test_data_nolab)  
test2_data_nolab.info()

customer_zip_code_prefix
seller_zip_code_prefix
order_weekday
order_day
order_month
<class 'autogluon.tabular.task.tabular_prediction.dataset.TabularDataset'>
RangeIndex: 13436 entries, 0 to 13435
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   customer_zip_code_prefix         13436 non-null  float64
 1   customer_city                    13436 non-null  object 
 2   customer_state                   13436 non-null  object 
 3   price                            13436 non-null  float64
 4   freight_value                    13436 non-null  float64
 5   product_weight_g                 13436 non-null  float64
 6   product_category_name_english    13436 non-null  object 
 7   seller_zip_code_prefix           13436 non-null  float64
 8   seller_city                      13436 non-null  object 
 9   seller_state                     13436 non-null  object 
 10  order_weekday        

In [16]:
# predictor = task.load(dir)  # unnecessary, just demonstrates how to load previously-trained predictor from file

# prediction = predictor.predict(X_test.to_csv(index=False))
# prediction = predictor.predict(test_data_nolab[0:2])
prediction = predictor.predict(test2_data_nolab[100:101])
print("Predictions:  ", prediction)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=prediction, auxiliary_metrics=True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

아래는 5개의 레이블에 대한 예측 확률을 보여 줌.
- API URL: https://auto.gluon.ai/api/autogluon.task.html#module-autogluon.tabular.tabular_prediction

In [ ]:
prediction_prob = predictor.predict_proba(test_data_nolab[0:1])
prediction_prob

## 리더 보드 생성
오토글루온에서 사용된 알고리즘의 성능 관련 지표를 확인 합니다. (예: score_test)

In [ ]:
predictor.leaderboard(test_data, silent=True)

## 모델 평가 및 Confusion 매트릭스
- SKLearn으로 accuracy, f1 score를 계산 합니다. 오토글루온의 수치와 동일한 결과가 나옵니다.
- Confusion 매트릭스 를 통해서 어느 레이블에서 잘 예측을 했는지를 확인 합니다.

In [ ]:
print(y_test)
print(prediction)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(y_test, prediction)
f1_score = f1_score(y_test, prediction, average='macro')
print("accuracy: %.2f%%" % accuracy)
print("f1_score: %.2f%%" % f1_score)

In [ ]:
from p_utils import plot_conf_mat


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

cm = confusion_matrix(y_true= y_test, y_pred= prediction)

plt.figure()
fig, ax = plt.subplots(figsize=(10,5))
plot_conf_mat(cm, 
              classes=['0','1', '2', '3', '4'], 
              title='Confusion Matrix')
plt.show()

## Feature Importance
- 피쳐의 중요도를 확인 합니다.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

fea_importance = predictor.feature_importance(test_data)

f, ax = plt.subplots(figsize=(10,5))
plot = sns.barplot(x=fea_importance.index, y = fea_importance.values)
# plot = sns.barplot(x=features, y= fea_importance)
ax.set_title('Feature Importance')
plot.set_xticklabels(plot.get_xticklabels(),rotation='vertical')
plt.show()